In [ ]:
import requests
import pyotp

base_url = "http://localhost:14374"
login_response = requests.post(
    f"{base_url}/register",
    json={
        "email": "test@test.com",
        "first_name": "Test",
        "last_name": "Name",
        "company_name": "Test Company",
        "job_title": "Test Job Title",
    },
)
response = login_response.json()
if "mfa_token" in response:
    mfa_token = response["mfa_token"]
    totp = pyotp.TOTP(mfa_token)
    print(f"Success! Your MFA token is: {totp.now()}")
else:
    print(response)